__Part 1: Set up and Pre-analysis__<br>
First initialize packages, load the Ecoinvent 3.4 database (https://www.ecoinvent.org/database/older-versions/ecoinvent-34/ecoinvent-34.html), and setup the technology matrix. We'll rely heavily on the brightway2 LCA package - in fact I think this algorithm is a great illustration of how the brightway2 package allows for creative utilization of LCA data. See for details: https://brightwaylca.org/

In [1]:
from brightway2 import *
import brightway2 as bw
import numpy as np
import scipy as s
import pandas as pd
import os
bw2setup()

Biosphere database already present!!! No setup is needed


In [2]:
projects.set_current("ecoinvent_import2")
list(databases)

['biosphere3', 'ecoinvent 3.4 cutoff']

In [3]:
db = Database("ecoinvent 3.4 cutoff")

Prior to the analysis we created the 'matching_indust' csv. This file is necessary for linking the economic data to the LCA data. For each product our dataset, we matched it to an Indian National Industry Classification (NIC) code, an ecoinvent process, and an Indian National Product Classification (NPC) code, using the researchers best judgement. It also includes a conversion factor allowing us to compare the units from the NPC data to ecoinvent process units. It also has an industry (NIC) specific capital to sales ratio. The capital to product sales ratios were calculated from Table 5 of: http://mospi.nic.in/asi-summary-results/104 (Value of Output/Invested Capital)

In [4]:
matches = pd.read_csv('matching_indust.csv')
matches.head()

,Product,NIC3,NIC3 code,Capital2Sales,codes,Activity Names,Ref Flow Names,location,amounts,Units,NPC_code,NPC_name,NPC_Unit,Conversion Factor
0,AC drives & Electrical Panels,Manufacture of other electrical equipment,279,1.472301,a169c23725cd8331377c288c6801b0bd,"electronics production, for control units","electronics, for control units",RoW,1,kilogram,4621299,Electrical apparatus for switching or protecti...,NOS,0.453592
1,Agarabathi,Manufacture of non-metallic mineral products n...,239,0.740689,1b77ea93d95ebea906450de98b493246,hard coal briquettes production,hard coal briquettes,RoW,1,megajoule,3533101,Agarbati,TH.NOS,628.000000
2,Agarabhattis,Manufacture of non-metallic mineral products n...,239,0.740689,1b77ea93d95ebea906450de98b493246,hard coal briquettes production,hard coal briquettes,RoW,1,megajoule,3533101,Agarbati,TH.NOS,628.000000
3,Agarbathies,Manufacture of non-metallic mineral products n...,239,0.740689,1b77ea93d95ebea906450de98b493246,hard coal briquettes production,hard coal briquettes,RoW,1,megajoule,3533101,Agarbati,TH.NOS,628.000000
4,Alluminium,Manufacture of basic precious and other non-fe...,242,1.642921,b416a8347ba3dbf0c8a1101f9efe5d4e,"sheet rolling, aluminium","sheet rolling, aluminium",RoW,1,kilogram,4143102,Aluminium ingots,T,1000.000000


Now create the LCI matrix. Each brightway/ecoinvent code observed in our economic data will be one column, and the rows will be the technosphere inputs and outputs pulled from the LCA data.

In [5]:
codes = matches['codes']
codes = list(set(codes)) # set of unique product codes
Tech = [] # Tech will be all codes in the ecoinvent database
for act in db:
    Tech.append(act.get('code'))
inputs = pd.DataFrame(0, index=Tech, columns=codes) # initialize an empty dataframe

In [6]:
## place flows in proper places in the LCI matrix

for i, c in enumerate(codes):
    temp1 = []
    temp2 = []
    act = db.get(codes[i])
    for exc in act.technosphere():
        temp1.append(exc.amount)
        temp2.append(exc.input['code'])
    inputs.loc[temp2, codes[i]] = temp1

In [7]:
inputs = inputs.loc[(inputs!=0).any(axis=1)]         ### delete any rows in which all values are zero
inputs.shape

(842, 132)

We also create an index csv that will useful for looking up the activities by code.

In [8]:
inputs_T = inputs.T
colnames = [db.get(c)['name'] for c in inputs_T.columns]
index = pd.DataFrame({'codes': inputs_T.columns, 'activities': colnames})
units = [db.get(c)['unit'] for c in inputs_T.columns]
index['units'] = units
index.to_csv('index.csv')
index.head()

,codes,activities,units
0,5093117becc60c17aa11cf03dbc4504b,"market for hazardous waste, for incineration",kilogram
1,6a81b4917d429988eba09940a97df970,"market for electricity, high voltage",kilowatt hour
2,fbdc7ab4413e89ef6589191cdc68cd42,market for spent solvent mixture,kilogram
3,cedc1e3baa442b21090333fd89dd2b33,market for lignite briquettes,megajoule
4,13cba3d573fc1151b28826bc9d8b784b,"market group for heat, district or industrial,...",megajoule


__Part 2: Calculating Local Production and Demand__<br>
We will represent the collective production of all the firms in Mysore as a demand vector. This means estimating the production mass of all the firms in our original dataset. Since our firm-level data is confidential, we'll simply upload the aggregated data here. The basic process was:
1. First calculate an estimate of sales using the capital data we had, and the sales2capital ratio from the matching_indust data
2. Calculate the average production mass per dollar using India's Annual Survey of Industries (ASI) microdata (Block J) for each NPC. This data is in the repository as 'ASI_blkj_summary.csv'
3. Convert NPC units to Ecoinvent units using our conversion factor. 
4. Merge the firm production masses to the Inputs_T data frame by ecoinvent code. Multiply each input or byproduct in the Inputs_T data by the production mass to appropriately scale demand for each firm, and then sum the inputs and byproducts across all firms in the data. This can also be done by subgroups - for example, we have summed demand by size of firms and also for firms that are in the main city. <br>
<br>
The result is the sums.csv file. Negative values indicate byproducts.


In [9]:
avg_mass = pd.read_csv('ASI_blkj_summary.csv')
avg_mass.rename(columns={'Product':'NPC_name'}, inplace = True)
avg_mass.head()

,Item Code,Quantity,Price,Rate,NPC_name,Unit
0,1101001,"44,591","548,590,299",12302.71353,Anthracite (raw coal),T
1,1101002,"1,487,376","7,763,476,546",5219.579008,Coal,T
2,1101003,"5,127,945","15,493,170,501",3021.321504,Coal (under sized),T
3,1101004,"1,753,650","1,486,271,507",847.5302979,Coal ash,T
4,1101005,"7,222,049","16,911,648,455",2341.669027,Coal compressed (middlings),T


In [10]:
sums = pd.read_csv('sums.csv')
sums.head()

,Unnamed: 0,codes,activities,units,Totals,Small,Medium,Large,Mysore City
0,0,e079bc1a542536f8154d5eb43dd64d36,market for scrap copper,kilogram,-1.226752e+05,-1.226752e+05,0.000000e+00,0.000000e+00,-1.226752e+05
1,1,1c55712379e5f46b4c997abe7adf911e,market for white spirit,kilogram,5.330248e+09,1.591700e+08,9.582318e+03,5.171069e+09,5.320403e+09
2,2,fbc5d4f66ca63fc47832691826277d7a,"market for heat, district or industrial, natur...",megajoule,1.978126e+09,1.949014e+09,4.284521e+06,2.482688e+07,1.949438e+09
3,3,d1693f29bd81ab2644398ba65efedb7c,"market for powder coat, aluminium sheet",square meter,3.766159e+05,3.766159e+05,0.000000e+00,0.000000e+00,3.765859e+05
4,4,b2ed3b3926bb5864851323822cb74f11,market for used motor scooter,unit,-4.572059e+05,0.000000e+00,0.000000e+00,-4.572059e+05,-4.572059e+05


__Part 3: Byproducts__ <br>
From the results dataframe, we gathered a list of all byproducts that appear in our system and systematically researched existing examples of industrial symbiosis in which the byproduct was reused. The result is the byproducts.xlsx spreadsheet. We need to rearrange this data slightly to get it ready for analysis. This section calculates the amount of each byproduct that is available, and the amount that could be demanded for use in each of the alternative substitutes. The large loop is due to the fact that some byproducts have multiple options for reuse, whereas others have fewer alternatives. <br>
<br>
We also calculate life cycle CO2e for each substitute, byproduct, and treatment process, making sure to reconcile reference flows to refer to the quantity of demand replaced. For substitutes the demand amount is equal to 1. For byproducts the demand amount is equal to -1 divided by the conversion factor. For the treatments, the demand amount depends on the specification of the reference flow, but is always converted into units consistent with the substitute. 

In [11]:
byproducts = pd.read_excel('byproducts.xlsx')
byproducts = byproducts[pd.isna(byproducts['Substitute Code 1'])==False] ## get rid of byproducts with no substitutes
idx = list(range(len(byproducts))) ## reset the index
byproducts['idx'] = idx
byproducts = byproducts.set_index('idx')
byproducts.head()

,Code (First),byproduct,Amount,Ref Units,Disposal Code,Substitute Code 1,Sub Name 1,Units 1,Conversion Factor 1,Treatment 1,...,Units 6,Conversion Factor 6,Treatment 6,Substitute Code 7,Sub Name 7,Units 7,Conversion Factor 7,Treatment 7,Example,Notes
idx,,,,,,,,,,,,,,,,,,,,,
0,4af142704238ac38488c57bc3bdd841a,"aluminium scrap, new, Recycled Content cut-off",-3.093217e+05,kilogram,4cce332d403c9966bcf032558d9b7df5,862553310bb68e1e2514f95f309c232c,"market for aluminium scrap, post-consumer",kilogram,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.sciencedirect.com/science/article/...,NaN
1,36a507490498ff58f985e11b59ed29bc,"iron scrap, unsorted, Recycled Content cut-off",-1.364894e+06,kilogram,9a919457b1512e4cb0627f524f9a631b,ece532f8286133279aaee49ba12cbbfc,"market for iron scrap, unsorted",kilogram,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://onlinelibrary.wiley.com/doi/epdf/10.11...,NaN
2,43a341f3ad04d3a542a910a2e4e4e372,market for ash from deinking sludge,-4.130968e+05,kilogram,bb024db5c3f5cf22571c4e6775e132c9,ec6f03e8f121139fb262f7c82912237c,market for sand,kilogram,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.sciencedirect.com/science/article/...,NaN
3,eb504f4cd619c009497963bac6a9d38a,market for average incineration residue,-4.821814e+05,kilogram,bb024db5c3f5cf22571c4e6775e132c9,ec6f03e8f121139fb262f7c82912237c,market for sand,kilogram,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.sciencedirect.com/science/article/...,NaN
4,04a35152bc7d9eb1dc15b149eef855ed,market for basic oxygen furnace waste,-5.347057e+03,kilogram,4ec883c43b51c8270b041e0aed32d305,92e387e281a6aab351faf6cedbfce012,"market for cement, Portland",kilogram,0.8,786a4cc05c6ddb989a823e63f0a44809,...,kilogram,0.8,786a4cc05c6ddb989a823e63f0a44809,NaN,NaN,NaN,NaN,NaN,https://www.sciencedirect.com/science/article/...,NaN


In [12]:
substitutes = []
byprod = []
sub_avail = []
byprod_avail = []
conversion = []
treatment = []
for i, b in enumerate(byproducts['byproduct']):
    if pd.isna(byproducts.loc[i, 'Substitute Code 1']) == False:
        substitutes.append(byproducts.loc[i, 'Substitute Code 1'])
        amt = np.multiply(byproducts.loc[i, 'Amount'], byproducts.loc[i, 'Conversion Factor 1'])
        sub_avail.append(-amt)
        byprod.append(byproducts.loc[i, 'Code (First)'])
        byprod_avail.append(byproducts.loc[i, 'Amount']*-1)
        conversion.append(byproducts.loc[i, 'Conversion Factor 1'])
        treatment.append(byproducts.loc[i, 'Treatment 1'])
        if pd.isna(byproducts.loc[i, 'Substitute Code 2']) == False:
            substitutes.append(byproducts.loc[i, 'Substitute Code 2'])
            amt = np.multiply(byproducts.loc[i, 'Amount'], byproducts.loc[i, 'Conversion Factor 2'])
            sub_avail.append(-amt)
            byprod.append(byproducts.loc[i, 'Code (First)'])
            byprod_avail.append(byproducts.loc[i, 'Amount']*-1)
            conversion.append(byproducts.loc[i, 'Conversion Factor 2'])
            treatment.append(byproducts.loc[i, 'Treatment 2'])
            if pd.isna(byproducts.loc[i, 'Substitute Code 3']) == False:
                substitutes.append(byproducts.loc[i, 'Substitute Code 3'])
                amt = np.multiply(byproducts.loc[i, 'Amount'], byproducts.loc[i, 'Conversion Factor 3'])
                sub_avail.append(-amt)
                byprod.append(byproducts.loc[i, 'Code (First)'])
                byprod_avail.append(byproducts.loc[i, 'Amount']*-1)
                conversion.append(byproducts.loc[i, 'Conversion Factor 3'])
                treatment.append(byproducts.loc[i, 'Treatment 3'])
                if pd.isna(byproducts.loc[i, 'Substitute Code 4']) == False:
                    substitutes.append(byproducts.loc[i, 'Substitute Code 4'])
                    amt = np.multiply(byproducts.loc[i, 'Amount'], byproducts.loc[i, 'Conversion Factor 4'])
                    sub_avail.append(-amt)
                    byprod.append(byproducts.loc[i, 'Code (First)'])
                    byprod_avail.append(byproducts.loc[i, 'Amount']*-1)
                    conversion.append(byproducts.loc[i, 'Conversion Factor 4'])
                    treatment.append(byproducts.loc[i, 'Treatment 4'])
                    if pd.isna(byproducts.loc[i, 'Substitute Code 5']) == False:
                        substitutes.append(byproducts.loc[i, 'Substitute Code 5'])
                        amt = np.multiply(byproducts.loc[i, 'Amount'], byproducts.loc[i, 'Conversion Factor 5'])
                        sub_avail.append(-amt)
                        byprod.append(byproducts.loc[i, 'Code (First)'])
                        byprod_avail.append(byproducts.loc[i, 'Amount']*-1)
                        conversion.append(byproducts.loc[i, 'Conversion Factor 5'])
                        treatment.append(byproducts.loc[i, 'Treatment 5'])
                        if pd.isna(byproducts.loc[i, 'Substitute Code 6']) == False:
                            substitutes.append(byproducts.loc[i, 'Substitute Code 6'])
                            amt = np.multiply(byproducts.loc[i, 'Amount'], byproducts.loc[i, 'Conversion Factor 6'])
                            sub_avail.append(-amt)
                            byprod.append(byproducts.loc[i, 'Code (First)'])
                            byprod_avail.append(byproducts.loc[i, 'Amount']*-1)
                            conversion.append(byproducts.loc[i, 'Conversion Factor 6'])
                            treatment.append(byproducts.loc[i, 'Treatment 6'])
                            if pd.isna(byproducts.loc[i, 'Substitute Code 7']) == False:
                                substitutes.append(byproducts.loc[i, 'Substitute Code 7'])
                                amt = np.multiply(byproducts.loc[i, 'Amount'], byproducts.loc[i, 'Conversion Factor 7'])
                                sub_avail.append(-amt)
                                byprod.append(byproducts.loc[i, 'Code (First)'])
                                byprod_avail.append(byproducts.loc[i, 'Amount']*-1)
                                conversion.append(byproducts.loc[i, 'Conversion Factor 7'])
                                treatment.append(byproducts.loc[i, 'Treatment 7'])
                            else:
                                continue
                        else:
                            continue
                    else:
                        continue
                else:
                    continue
            else:
                continue
        else:
            continue  
    else:
        continue

In [13]:
substitutes_set = list(set(substitutes))
sub_amounts = [float(sums[sums['codes']==s]['Totals']) for s in substitutes_set]
lookup = pd.DataFrame({'substitutes': substitutes_set, 'demand_tot': sub_amounts})
lookup.head()

,substitutes,demand_tot
0,04c0b1b7d1ad8bffaec0824dad10684f,3.107193e+06
1,f73ebbe3fe8fd9ac4f9f5547726a5386,1.229789e+03
2,6ca69ad08a4b9d24b23277c4f2b81bd9,9.331506e+03
3,13c3413a7ddca42350133727eac1b22d,1.369614e+03
4,d4d40c24ff4c9e01f4c743adf8fc99e7,7.775442e+06


In [14]:
byprod_results = pd.DataFrame({'substitutes': substitutes, 'supply': sub_avail, 'byprod': byprod, 'byprod_avail': byprod_avail, 'conversion': conversion, 'treatment': treatment})
byprod_results['sub_demand_tot'] = [float(lookup[lookup['substitutes'] == s]['demand_tot']) for s in byprod_results['substitutes']]
byprod_results = byprod_results[['substitutes', 'sub_demand_tot', 'supply', 'byprod', 'byprod_avail', 'conversion', 'treatment']]
byprod_results.head()

,substitutes,sub_demand_tot,supply,byprod,byprod_avail,conversion,treatment
0,862553310bb68e1e2514f95f309c232c,1.333426e+06,3.093217e+05,4af142704238ac38488c57bc3bdd841a,3.093217e+05,1.0,NaN
1,ece532f8286133279aaee49ba12cbbfc,4.115240e+06,1.364894e+06,36a507490498ff58f985e11b59ed29bc,1.364894e+06,1.0,NaN
2,ec6f03e8f121139fb262f7c82912237c,2.677721e+05,4.130968e+05,43a341f3ad04d3a542a910a2e4e4e372,4.130968e+05,1.0,NaN
3,0f849d9fbaff07c9619cff16b05774dd,1.153391e+05,4.130968e+05,43a341f3ad04d3a542a910a2e4e4e372,4.130968e+05,1.0,NaN
4,d82156c2de5c3a07fc6d87490112ba2f,4.092338e+05,4.130968e+05,43a341f3ad04d3a542a910a2e4e4e372,4.130968e+05,1.0,NaN


This section calculates the lifecycle CO2e emissions associated with each substitute, byproduct, and treatment process

In [15]:
substituteCCs = []
for i in byprod_results['substitutes']:
    act = db.get(i)
    amt = 1
    LCAi_CC = bw.LCA({act: amt}, ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'))
    LCAi_CC.lci()
    LCAi_CC.lcia()
    substituteCCs.append(LCAi_CC.score)
byprod_results['substituteCCs'] = substituteCCs

In [16]:
bypCCs = []
for b, c in zip(byprod_results['byprod'], byprod_results['conversion']):
    act = db.get(b)
    amt = -1 ## np.divide(-1, c)
    LCAi_CC = bw.LCA({act: amt}, ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'))
    LCAi_CC.lci()
    LCAi_CC.lcia()
    bypCCs.append(LCAi_CC.score)
byprod_results['bypCCs'] = bypCCs

I also created this treatment index by looking up the codes for the treatment processes. The exception column refers to the fact that for byproducts that have multiple potential uses, some of the uses have different units in ecoinvent, so different conversion factors need to be applied. 

In [17]:
treatmentidx = pd.read_csv('treatmentidx.csv') 
treatmentidx.head()

,treatment,activity,unit,ref flow,amt,exception
0,6116ebff20b1cd36f16e1e93be27032f,"treatment of waste wood, post-consumer, sortin...",kg,"wood chips, from post-consumer wood, measured ...",1.0,NaN
1,bdf31e4a628048eac31515cc0baae678,aluminium oxide production,kg,aluminium oxide kilogram 1.0,1.0,NaN
2,2e04a1d722ba85acffebacf2ef3090e5,"treatment of aluminium scrap, new, at refiner",kg,"aluminium, cast alloy kilogram 1.0",1.0,NaN
3,786a4cc05c6ddb989a823e63f0a44809,ground granulated blast furnace slag production,kg,ground granulated blast furnace slag kilogram 1.0,1.0,NaN
4,20f8fd229472f0f624c716596dd2e54e,treatment of copper scrap by electrolytic refi...,kg,copper kilogram 1.0,1.0,NaN


In [18]:
treatmentCCs = []
treatmentamts = []
for t, s in zip(byprod_results['treatment'], byprod_results['substitutes']):
    try:
        act = db.get(t)
        tempidx = treatmentidx[treatmentidx['treatment']==t]
        amt = tempidx['amt'].tolist()[0]
        LCAi_CC = bw.LCA({act: amt}, ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'))
        LCAi_CC.lci()
        LCAi_CC.lcia()
        treatmentCCs.append(LCAi_CC.score)
        treatmentamts.append(amt)
    except:
        treatmentCCs.append(0)
        treatmentamts.append(0)
byprod_results['treatmentCCs'] = treatmentCCs
byprod_results['treatmentamts'] = treatmentamts
byprod_results.head()

,substitutes,sub_demand_tot,supply,byprod,byprod_avail,conversion,treatment,substituteCCs,bypCCs,treatmentCCs,treatmentamts
0,862553310bb68e1e2514f95f309c232c,1.333426e+06,3.093217e+05,4af142704238ac38488c57bc3bdd841a,3.093217e+05,1.0,NaN,0.034410,0.000000,0.0,0.0
1,ece532f8286133279aaee49ba12cbbfc,4.115240e+06,1.364894e+06,36a507490498ff58f985e11b59ed29bc,1.364894e+06,1.0,NaN,0.003105,0.000000,0.0,0.0
2,ec6f03e8f121139fb262f7c82912237c,2.677721e+05,4.130968e+05,43a341f3ad04d3a542a910a2e4e4e372,4.130968e+05,1.0,NaN,0.011989,0.350702,0.0,0.0
3,0f849d9fbaff07c9619cff16b05774dd,1.153391e+05,4.130968e+05,43a341f3ad04d3a542a910a2e4e4e372,4.130968e+05,1.0,NaN,0.012061,0.350702,0.0,0.0
4,d82156c2de5c3a07fc6d87490112ba2f,4.092338e+05,4.130968e+05,43a341f3ad04d3a542a910a2e4e4e372,4.130968e+05,1.0,NaN,0.010565,0.350702,0.0,0.0


In [19]:
totalCCs = np.array(byprod_results['substituteCCs']) + np.divide(np.array(byprod_results['bypCCs']), np.array(byprod_results['conversion'])) - np.array(byprod_results['treatmentCCs'])
byprod_results['totalCCs'] = totalCCs

As hinted at above, units are really tricky here! Since the units for three of the treatment processes had different ecoinvent units depending on what they substitute for, we had to apply the appropriate conversion factors. I just did it manually below.

In [20]:
byprod_results.loc[129, 'totalCCs'] = byprod_results.loc[129, 'substituteCCs'] + (byprod_results.loc[129, 'bypCCs']/byprod_results.loc[129, 'conversion']) - (byprod_results.loc[129, 'treatmentCCs']/.06173)
byprod_results.loc[92, 'totalCCs'] = byprod_results.loc[92, 'substituteCCs'] + (byprod_results.loc[92, 'bypCCs']/byprod_results.loc[92, 'conversion']) - (byprod_results.loc[92, 'treatmentCCs']/500)
byprod_results.loc[93, 'totalCCs'] = byprod_results.loc[93, 'substituteCCs'] + (byprod_results.loc[93, 'bypCCs']/byprod_results.loc[93, 'conversion']) - (byprod_results.loc[93, 'treatmentCCs']/500)

__Part 4: Symbiosis Potential__ <br>
The Symbiosis Potential algorithm will run through each potential exchange in the order that they appear in the spreadsheet. Therefore the spreadsheet can be ordered to optimize for different environmental targets. In this case the algorithm will be optimized for CO2e reductions. <br>
<br>
Because it can be confusing - let's go over the column definitions and units:<br>
sub_demand_tot: quantity of the substitute demanded in units of the substitute<br>
supply: quantity of the byproduct available in units of the substitute<br>
byprod_avail: quantity of byproduct available in units of the byproduct<br>
supply_remaining: quantity of byproduct left over after symbiosis in units of the substitute<br>
demand_remaining: quantity of the substitute left over in units of the substitute<br>
demand_met: quantity used in units of the substitute<br>

In [21]:
byprod_results = byprod_results.sort_values(by='totalCCs', ascending=False)
byprod_results = byprod_results.rename_axis('index_old').reset_index()
byprod_results.head()

,index_old,substitutes,sub_demand_tot,supply,byprod,byprod_avail,conversion,treatment,substituteCCs,bypCCs,treatmentCCs,treatmentamts,totalCCs
0,130,d4883bb611557107b44ed2ccc70bc3c7,4015.937688,274.269741,4956343d45f1e4da6d6c635bd17a7f3f,3.428372e+03,0.080000,811fabc636b507d0d6f390582fc957b3,343.650374,9.207618,0.000000,1.0,458.745593
1,64,8bae8bac48e6f42f88bf74c08199b681,5872.689139,102.575920,466bf4344a5a3c72e47ecb0aa43b26a2,1.025759e+05,0.001000,f593e681dd3b8e3ea76f17cebd28aeeb,1.929310,0.177145,0.000000,1.0,179.074321
2,63,f91efb8758ca203688a82efaada469d5,242261.288362,102.852875,466bf4344a5a3c72e47ecb0aa43b26a2,1.025759e+05,0.001003,80e5d1a7a269279e82e3f86271b0d6e7,0.471473,0.177145,0.000000,1.0,177.139480
3,93,cfbc8a71f3cec9b8e593470d411cb18f,19177.957502,18303.259739,8d6425d6f85ce7abe890366b6c4749aa,9.151630e+06,0.002000,c03201092837c82f4a0fa39214ff7431,35.040881,0.019138,0.060582,1.0,44.609930
4,128,cfbc8a71f3cec9b8e593470d411cb18f,19177.957502,2135.329800,09bb2cc9ff96eb23d4f6523035e410fe,1.067665e+06,0.002000,6116ebff20b1cd36f16e1e93be27032f,35.040881,0.016165,0.019427,1.0,43.104177


In [22]:
supply_remaining = byprod_results['supply'].copy()
demand_met = []
demand_remaining = byprod_results['sub_demand_tot'].copy()
pd.options.mode.chained_assignment = None
for i, b in enumerate(byprod):
    demand_met.append(min(demand_remaining[i], supply_remaining[i]))
    if supply_remaining[i] >= demand_remaining[i]:
        supply_remaining.loc[byprod_results['byprod'] == byprod_results.loc[i, 'byprod']] = np.multiply(((supply_remaining[i] - demand_met[i]) / byprod_results.loc[i, 'conversion']), byprod_results.loc[byprod_results['byprod'] == byprod_results.loc[i, 'byprod'], 'conversion']) 
        demand_remaining.loc[byprod_results['substitutes'] == byprod_results.loc[i, 'substitutes']] = 0
    else:
        supply_remaining.loc[byprod_results['byprod'] == byprod_results.loc[i, 'byprod']] = 0
        demand_remaining.loc[byprod_results['substitutes'] == byprod_results.loc[i, 'substitutes']] = demand_remaining[i] - demand_met[i]  

byprod_results['supply_remaining'] = supply_remaining
byprod_results['demand_remaining'] = demand_remaining
byprod_results['demand_met'] = demand_met
byprod_results.to_csv('byprod_results.csv')
byprod_results.head()    

,index_old,substitutes,sub_demand_tot,supply,byprod,byprod_avail,conversion,treatment,substituteCCs,bypCCs,treatmentCCs,treatmentamts,totalCCs,supply_remaining,demand_remaining,demand_met
0,130,d4883bb611557107b44ed2ccc70bc3c7,4015.937688,274.269741,4956343d45f1e4da6d6c635bd17a7f3f,3.428372e+03,0.080000,811fabc636b507d0d6f390582fc957b3,343.650374,9.207618,0.000000,1.0,458.745593,0.000000,3741.667947,274.269741
1,64,8bae8bac48e6f42f88bf74c08199b681,5872.689139,102.575920,466bf4344a5a3c72e47ecb0aa43b26a2,1.025759e+05,0.001000,f593e681dd3b8e3ea76f17cebd28aeeb,1.929310,0.177145,0.000000,1.0,179.074321,0.000000,5770.113219,102.575920
2,63,f91efb8758ca203688a82efaada469d5,242261.288362,102.852875,466bf4344a5a3c72e47ecb0aa43b26a2,1.025759e+05,0.001003,80e5d1a7a269279e82e3f86271b0d6e7,0.471473,0.177145,0.000000,1.0,177.139480,0.000000,0.000000,0.000000
3,93,cfbc8a71f3cec9b8e593470d411cb18f,19177.957502,18303.259739,8d6425d6f85ce7abe890366b6c4749aa,9.151630e+06,0.002000,c03201092837c82f4a0fa39214ff7431,35.040881,0.019138,0.060582,1.0,44.609930,0.000000,0.000000,18303.259739
4,128,cfbc8a71f3cec9b8e593470d411cb18f,19177.957502,2135.329800,09bb2cc9ff96eb23d4f6523035e410fe,1.067665e+06,0.002000,6116ebff20b1cd36f16e1e93be27032f,35.040881,0.016165,0.019427,1.0,43.104177,1099.561037,0.000000,874.697763


__Part 5: LCIA__ <br>
To do the LCIA, we will create two different demand vectors and compare the life cycle impacts resulting from each
<br><br>
demandvec1 - the baseline:<br>
1. Take substitutes and sub_demand_tot and eliminate duplicates - these are demanded inputs
2. Take byprod and byprod_avail and eliminate duplicates - these are demands for waste treatment processes. Multiply by -1

demandvec2 - with symbiosis:<br>
1. Take demand_remaining as demand for inputs
2. Take supply_remaining as demand for waste treatment processes (times -1), dividing by the conversion factor. 
3. Take demand_met as demand for treatment processes. Multiply by 1 or -1 depending on the treatment process reference flow. Exchanges that don't have a treatment process are considered to be burden free (cutoff approach). In three exchanges, because of different units and negative reference flows - the treatment process is applied to the amount the amount of demand replaced, but in the units of the byproduct (both cases of paperboard for pulpwood, and waste wood to woodchips). This is corrected manually below

In [23]:
demandvec1 = byprod_results[['substitutes', 'sub_demand_tot']]
demandvec1 = demandvec1.drop_duplicates()
demandvec1b = byprod_results[['byprod', 'byprod_avail']]
demandvec1b = demandvec1b.drop_duplicates()
demandvec1b.byprod_avail = demandvec1b.byprod_avail.multiply(-1)
demandvec1.columns = ['activity', 'amount']
demandvec1b.columns = ['activity', 'amount']
demandvec1 = pd.concat([demandvec1, demandvec1b])
demandvec1.head()

,activity,amount
0,d4883bb611557107b44ed2ccc70bc3c7,4015.937688
1,8bae8bac48e6f42f88bf74c08199b681,5872.689139
2,f91efb8758ca203688a82efaada469d5,242261.288362
3,cfbc8a71f3cec9b8e593470d411cb18f,19177.957502
5,6d656e138e0bda92b92375bf00aa749f,381591.564778


In [24]:
demandvec2 = byprod_results[['substitutes', 'demand_remaining']]
demandvec2 = demandvec2.drop_duplicates()
demandvec2_b = byprod_results[['byprod', 'supply_remaining', 'conversion']]
demandvec2_b = demandvec2_b.drop_duplicates() 
demandvec2_b['amount'] = np.divide(demandvec2_b['supply_remaining'], demandvec2_b['conversion'])
demandvec2_b.amount = demandvec2_b.amount.multiply(-1)
demandvec2_b = demandvec2_b[['byprod', 'amount']]
demandvec2_b = demandvec2_b.drop_duplicates() ### drop duplicates again b/c some of same have diff conversion factors...might have been an error with 09bb in original analysis
demandvec2_t = byprod_results[['index_old', 'supply_remaining', 'conversion', 'treatment', 'demand_met', 'treatmentamts']]
demandvec2_t['amount'] = np.multiply(demandvec2_t['demand_met'], demandvec2_t['treatmentamts'])
demandvec2_t.loc[demandvec2_t.index_old == 129, 'amount'] =  demandvec2_t.loc[demandvec2_t.index_old == 129, 'amount'] * demandvec2_t.loc[demandvec2_t.index_old == 129, 'conversion']
demandvec2_t.loc[demandvec2_t.index_old == 92, 'amount'] = demandvec2_t.loc[demandvec2_t.index_old == 92, 'amount'] * demandvec2_t.loc[demandvec2_t.index_old == 92, 'conversion']
demandvec2_t.loc[demandvec2_t.index_old == 93, 'amount'] = demandvec2_t.loc[demandvec2_t.index_old == 93, 'amount'] * demandvec2_t.loc[demandvec2_t.index_old == 93, 'conversion']
demandvec2_t = demandvec2_t[['treatment', 'amount']]
demandvec2_t = demandvec2_t.drop_duplicates()
demandvec2_t = demandvec2_t.groupby(['treatment'], as_index=False).sum()
demandvec2.columns = ['activity', 'amount']
demandvec2_b.columns = ['activity', 'amount']
demandvec2_t.columns = ['activity', 'amount']
demandvec2 = pd.concat([demandvec2, demandvec2_b, demandvec2_t])
demandvec2 = demandvec2[demandvec2.amount != 0]
demandvec2.head()

,activity,amount
0,d4883bb611557107b44ed2ccc70bc3c7,3.741668e+03
1,8bae8bac48e6f42f88bf74c08199b681,5.770113e+03
12,bde7404438fdfc7938698920aa9fc156,3.897707e+07
14,d526bb3cd54b6a4100208f40cc5ee59c,3.503862e+06
15,04c0b1b7d1ad8bffaec0824dad10684f,3.107193e+06


In [25]:
acts1 = [db.get(a) for a in demandvec1['activity']]
amts1 = demandvec1['amount'].tolist()
amts1 = [float(a) for a in amts1]
acts2 = [db.get(a) for a in demandvec2['activity']]
amts2 = demandvec2['amount'].tolist()
amts2 = [float(a) for a in amts2]
demand_vector1 = dict(zip(acts1, amts1))
demand_vector2 = dict(zip(acts2, amts2))

In [26]:
LCA1_HT = bw.LCA(demand_vector1, ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'))
LCA2_HT = bw.LCA(demand_vector2, ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'))
LCA1_HT.lci()
LCA1_HT.lcia()
LCA2_HT.lci()
LCA2_HT.lcia()
print(LCA1_HT.score, LCA2_HT.score)

484413821452.97925 484395510408.1822


In [27]:
LCA1_CC = bw.LCA(demand_vector1, ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'))
LCA2_CC = bw.LCA(demand_vector2, ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'))
LCA1_CC.lci()
LCA1_CC.lcia()
LCA2_CC.lci()
LCA2_CC.lcia()
print(LCA1_CC.score, LCA2_CC.score)

11221134014.934225 11147458246.836199


In [28]:
LCA1_PM = bw.LCA(demand_vector1, ('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'))
LCA2_PM = bw.LCA(demand_vector2, ('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'))
LCA1_PM.lci()
LCA1_PM.lcia()
LCA2_PM.lci()
LCA2_PM.lcia()
print(LCA1_PM.score, LCA2_PM.score)

260260831.78242356 260164317.17527506


In [29]:
LCA1_HT.score - LCA2_HT.score

18311044.797058105

In [30]:
LCA1_CC.score - LCA2_CC.score

73675768.09802628

In [31]:
LCA1_PM.score - LCA2_PM.score

96514.6071484983